In [1]:
from statistics import mode
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertForTokenClassification
from torch.utils.data import DataLoader
from tools.data_utils import get_unique_tags
from tools.data import SequenceLabelingDataset, collate_fn
from tools.data_utils import read_jsonl_file
from torch.utils.data import RandomSampler, SequentialSampler
from train_and_eval import Trainer
import torch
from config import Config

# 加载数据
train_data = read_jsonl_file('./data/cluener/conll_format/train.jsonl')
dev_data = read_jsonl_file('./data/cluener/conll_format/dev.jsonl')

# 得到标签相关信息, 一方面为了将标签转换为索引, 另一方面为BERT初始化时使用
unique_tags, labels_to_ids, ids_to_labels = get_unique_tags(train_data)

# tokenizer用于处理文本
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

train_dataset = SequenceLabelingDataset(data=train_data, labels_to_ids=labels_to_ids, tokenizer=tokenizer, max_length=128)
dev_dataset = SequenceLabelingDataset(data=dev_data, labels_to_ids=labels_to_ids, tokenizer=tokenizer, max_length=128)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, collate_fn=collate_fn)

dev_sampler = SequentialSampler(dev_dataset)
dev_dataloader = DataLoader(dev_dataset, batch_size=32, sampler=dev_sampler, collate_fn=collate_fn)

# 模型
model = BertForTokenClassification.from_pretrained("bert-base-chinese", num_labels=len(unique_tags))

# 超参数等设置
opt = Config()
# 设置设备, 数据已经移动到config中设置的设备上
# 因此需要将模型也移动到相同设备上
model = model.to(opt.device)
trainer = Trainer(model=model, ids2labels=ids_to_labels, opt=opt)

/home/hejiansu/anaconda3/envs/deepner/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification f

In [2]:
for i in range(opt.epochs):
    print(f"epoch: {i}")
    trainer.train(data_loader=train_dataloader, epoch=i)
    results = trainer.eval(data_loader=dev_dataloader, epoch=i)

epoch: 0


  0%|          | 1/336 [00:00<00:58,  5.75it/s]

Train Loss: 3.0300095081329346


  1%|          | 2/336 [00:00<00:45,  7.35it/s]

Train Loss: 2.690946102142334


...

 99%|█████████▉| 333/336 [00:32<00:00, 10.97it/s]

Train Loss: 0.1430431604385376
Train Loss: 0.2296697050333023
Train Loss: 0.2574882209300995


100%|██████████| 336/336 [00:32<00:00, 10.21it/s]

Train Loss: 0.10322980582714081
Train Loss: 0.13559798896312714
Train Loss: 0.1907421201467514
Evaluating...


*****Evaluation Results in Epoch 1*****
 acc: 0.7805 - recall: 0.7619 - f1: 0.7711 - loss: 8.5829 
***** Entity results in Epoch 1 *****
address
 acc: 0.6143 - recall: 0.6177 - f1: 0.6160 
book
 acc: 0.8372 - recall: 0.7105 - f1: 0.7687 
company
 acc: 0.7717 - recall: 0.7760 - f1: 0.7738 
game
 acc: 0.8248 - recall: 0.7902 - f1: 0.8071 
government
 acc: 0.8008 - recall: 0.8074 - f1: 0.8041 
movie
 acc: 0.8025 - recall: 0.8400 - f1: 0.8208 
name
 acc: 0.8747 - recall: 0.8806 - f1: 0.8777 
organization
 acc: 0.7358 - recall: 0.7551 - f1: 0.7453 
position
 acc: 0.8095 - recall: 0.7618 - f1: 0.7849 
scene
 acc: 0.7669 - recall: 0.6281 - f1: 0.6906 


In [3]:
pred_results = trainer.predict(data_loader=dev_dataloader)

Predicting...


Predicting: 100%|██████████| 42/42 [00:02<00:00, 19.29it/s]


In [8]:
pred_results[0]

[['name', 0, 2], ['address', 15, 15]]